In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
import sklearn.metrics as f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold

In [2]:
bids_df= pd.read_csv('data/bids.csv')
train_df = pd.read_csv('data/train.csv')
test_df= pd.read_csv('data/test.csv')

In [3]:
bids_df

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3
...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,9709222052631578,ru,140.204.227.63,cghhmomsaxi6pug
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,9709222052631578,pl,24.232.159.118,wgggpdg2gx5pesn
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,9709222052631578,za,80.237.28.246,5xgysg14grlersa
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,9709222052631578,my,91.162.27.152,bhtrek44bzi2wfl


# Feature Engineering

In [4]:
bids_unique= bids_df.groupby('bidder_id').nunique()
bids_unique = bids_unique.drop(["bid_id"],axis = 1)

#### From bids_unique dataframe, we may be interested in the number of times a bidder has 
1. Bidded in a different auction
2. Bidded from a different url
3. Bidded on a different device
4. Bidded from a different country

In [5]:
bids_unique

,auction,merchandise,device,time,country,ip,url
bidder_id,,,,,,,
001068c415025a009fee375a12cff4fcnht8y,1,1,1,1,1,1,1
002d229ffb247009810828f648afc2ef593rb,1,1,2,2,1,1,1
0030a2dd87ad2733e0873062e4f83954mkj86,1,1,1,1,1,1,1
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3,1,3,3,1,3,2
00486a11dff552c4bd7696265724ff81yeo9v,13,1,8,20,1,10,7
...,...,...,...,...,...,...,...
ffbc0fdfbf19a8a9116b68714138f2902cc13,637,1,792,23487,102,18726,8039
ffc4e2dd2cc08249f299cab46ecbfacfobmr3,15,1,13,22,6,18,12
ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1,1,1,1,1,1,1


In [6]:
bids_df_count= bids_df.groupby("bidder_id")
counts= bids_df_count['url'].count().reset_index().rename(columns = {'url':'num_bids'})

#### From counts dataframe, we know the number of times that a unique bidder has performed a bid 

In [7]:
counts

,bidder_id,num_bids
0,001068c415025a009fee375a12cff4fcnht8y,1
1,002d229ffb247009810828f648afc2ef593rb,2
2,0030a2dd87ad2733e0873062e4f83954mkj86,1
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3
4,00486a11dff552c4bd7696265724ff81yeo9v,20
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,25075
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1
6612,ffd62646d600b759a985d45918bd6f0431vmz,664


#### Joining the new features together with the training dataset

In [8]:
new_train = train_df.merge(bids_unique, how= 'left', on = 'bidder_id')
new_train = new_train.merge(counts, how= 'left')
new_train.fillna(0,inplace = True)

new_test = test_df.merge(bids_unique, how= 'left', on = 'bidder_id')
new_test = new_test.merge(counts, how= 'left')
new_test.fillna(0,inplace = True)

#### Check that the size new training and test dataset is the same as the old training and test dataset

In [9]:
print("Size of old train: " + str(train_df.shape))
print("Size of new train: " + str(new_train.shape))
print("Size of old test: " + str(test_df.shape))
print("Size of new test: " + str(new_test.shape))

Size of old train: (2013, 4)
Size of new train: (2013, 12)
Size of old test: (4700, 3)
Size of new test: (4700, 11)


#### Count of number of first and last bids for each bidder

In [10]:
auctions = bids_df.groupby('auction')
first_bid_list = set(auctions.first()['bidder_id'])
last_bid_list = set(auctions.last()['bidder_id'])

In [11]:
# functions to add a counter if a particular bidder id is in the first and/or last bid
def check_first_bid(dummy):
    count = 0
    for bidder_id in first_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
def check_last_bid(dummy):
    count = 0
    for bidder_id in last_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count

In [12]:
num_first_bids_train = new_train['bidder_id'].apply(func = check_first_bid)
num_last_bids_train = new_train['bidder_id'].apply(func = check_last_bid)

num_first_bids_test = new_test['bidder_id'].apply(func = check_first_bid)
num_last_bids_test = new_test['bidder_id'].apply(func = check_last_bid)

In [13]:
# joining the new train and test with the number of first and last bids
new_train['num_first_bids'] = num_first_bids_train
new_train['num_last_bids'] = num_last_bids_train
new_train

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,url,num_bids,num_first_bids,num_last_bids
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,24.0,0,0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,3.0,0,0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,4.0,0,0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,155.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,5.0,2.0,36.0,1,0
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,0,0
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0


In [14]:
new_test['num_first_bids'] = num_first_bids_test
new_test['num_last_bids'] = num_last_bids_test
new_test

,bidder_id,payment_account,address,auction,merchandise,device,time,country,ip,url,num_bids,num_first_bids,num_last_bids
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c,3.0,1.0,2.0,4.0,3.0,4.0,3.0,4.0,0,0
1,a921612b85a1494456e74c09393ccb65ylp4y,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn,2.0,1.0,3.0,3.0,2.0,2.0,1.0,3.0,0,0
2,6b601e72a4d264dab9ace9d7b229b47479v6i,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0,14.0,1.0,4.0,17.0,3.0,4.0,2.0,17.0,0,0
3,eaf0ed0afc9689779417274b4791726cn5udi,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat,90.0,1.0,81.0,148.0,14.0,129.0,80.0,148.0,0,0
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa,20.0,1.0,17.0,23.0,2.0,17.0,1.0,23.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,41.0,1.0,9.0,466.0,5.0,22.0,4.0,983.0,0,0
4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,32.0,1.0,29.0,66.0,10.0,49.0,18.0,66.0,0,0
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,419.0,1.0,376.0,2156.0,86.0,1460.0,1049.0,2162.0,1,1
4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,5.0,1.0,4.0,5.0,1.0,5.0,2.0,5.0,0,0


In [15]:
#entropy
def log_entropy(x):
    e = np.sum(np.log(np.array(range(1,np.sum(x)))))
    for i in x:
        e -= np.sum(np.log(np.array(range(1,i))))
    return e

In [16]:
entropy_df = new_train.copy()
entropy_df['ip_entropy'] = 0
entropy_df['url_entropy'] = 0
# entropy_df
for bidder in entropy_df['bidder_id']:
    temp_df = bids_df[bids_df['bidder_id'] == bidder][['ip','url']]
    entropy_df.loc[entropy_df[entropy_df['bidder_id'] == bidder].index, ['ip_entropy', 'url_entropy']] = \
                    round(log_entropy(temp_df.ip.groupby(temp_df.ip).count()),4), \
                    round(log_entropy(temp_df.url.groupby(temp_df.url).count()),4)
    
entropy_df = entropy_df[['bidder_id', 'ip_entropy', 'url_entropy']]
entropy_df

,bidder_id,ip_entropy,url_entropy
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,49.8149,0.0000
1,624f258b49e77713fc34034560f93fb3hu3jo,0.6931,0.6931
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,1.7918,1.0986
3,4bee9aba2abda51bf43d639013d6efe12iycd,0.0000,0.0000
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,601.0291,460.9263
...,...,...,...
2008,369515b3af4f8ca582f90271d30b14b6r52aw,26.0870,9.8851
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,0.0000,0.0000
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,0.0000,0.0000
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,0.0000,0.0000


In [17]:
# fraction of IPs used by a bidder which were also used by another user which was a bot (jf)
#bids per ip used by bots
bot_or_human = train_df
ip_used_by_bot = pd.merge(bids_df, bot_or_human[['bidder_id', 'outcome']], on='bidder_id', how='left')

# #make outcome num a float num
ip_used_by_bot['outcome'] = 1.0 * (ip_used_by_bot['outcome'] == 1)

#get unique ip by each bidder 
unique_ip_per_bidder = ip_used_by_bot.groupby(['ip', 'bidder_id']).outcome.mean().reset_index() 

#identify ips that are a bot
bot_ips = unique_ip_per_bidder.groupby('ip').outcome.sum().reset_index() 
bot_ips['num_bots_on_ip'] = bot_ips.outcome
bids = pd.merge(ip_used_by_bot, bot_ips[['ip', 'num_bots_on_ip']], on='ip', how='left')

#find out bidders that are using ips used by bots
bids['has_ip_used_by_a_bot'] = 1 * ((bids['num_bots_on_ip'] - bids['outcome']) >= 1) 

#drop bidders who are already identified as bots in train dataset
bids = bids.drop('outcome', 1)

#group by unique ip and bidder
unique_ip_per_bidder = bids.groupby(['bidder_id', 'ip']).has_ip_used_by_a_bot.mean().reset_index() 

#get bidder that has ips used by a bot
bot_ips = unique_ip_per_bidder.groupby('bidder_id').has_ip_used_by_a_bot.mean().reset_index()

#final dataframe
bot_or_human = pd.merge(bot_or_human, bot_ips[['bidder_id', 'has_ip_used_by_a_bot']], on='bidder_id', how='left')
bot_or_human['on_ip_that_has_a_bot_mean'] = bot_or_human['has_ip_used_by_a_bot'].fillna(0)
bot_or_human['has_ip_used_by_a_bot'] = 1*(bot_or_human['has_ip_used_by_a_bot'].fillna(0) > 0)
bot_or_human = bot_or_human[['bidder_id', 'on_ip_that_has_a_bot_mean']]
bot_or_human

<ipython-input-17-49bb5d4940f1>:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bids = bids.drop('outcome', 1)


,bidder_id,on_ip_that_has_a_bot_mean
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,0.400000
1,624f258b49e77713fc34034560f93fb3hu3jo,0.000000
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,1.000000
3,4bee9aba2abda51bf43d639013d6efe12iycd,1.000000
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.260163
...,...,...
2008,369515b3af4f8ca582f90271d30b14b6r52aw,0.800000
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,0.000000
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,0.500000
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,1.000000


In [18]:
#get percentage_of_auctions_above_threshold
num_bidders_per_auction = bids_df.groupby("auction").nunique()
avg_num_bids_per_auction = num_bidders_per_auction['bid_id']/num_bidders_per_auction['bidder_id']
avg_num_bids_per_auction_df = avg_num_bids_per_auction.to_frame()
avg_num_bids_per_auction_df = avg_num_bids_per_auction_df.reset_index()
avg_num_bids_per_auction_df = avg_num_bids_per_auction_df.rename(columns={"auction": "auction", 0: "threshold"})
auction_df = bids_df.groupby(['bidder_id', 'auction']).size()
auction_df = pd.DataFrame(auction_df, columns = ['auction_bids_count'])
auction_df_reset = auction_df.reset_index()
auction_df_final = auction_df_reset.merge(avg_num_bids_per_auction_df, on='auction')

auction_df_final['exceeds_auction_bids_count_threshold'] = 1 * (auction_df_final['auction_bids_count'] > auction_df_final['threshold'])
total_no_of_participated_auctions_df = auction_df_final.groupby('bidder_id').nunique()
total_no_of_participated_auctions_df = total_no_of_participated_auctions_df[['auction']]
total_no_of_participated_auctions_df = total_no_of_participated_auctions_df.rename(columns={"auction":"total_no_of_participated_auctions"})

bidders_above_auction_threshold = auction_df_final[auction_df_final['exceeds_auction_bids_count_threshold'] == 1]
bidders_above_auction_threshold_df = bidders_above_auction_threshold.groupby('bidder_id').nunique()
bidders_above_auction_threshold_df = bidders_above_auction_threshold_df[['auction']]
bidders_above_auction_threshold_df = bidders_above_auction_threshold_df.rename(columns={"auction": "no_of_auction_exceeds_threshold"})

auction_bids = total_no_of_participated_auctions_df.merge(bidders_above_auction_threshold_df, how= 'left', on = 'bidder_id')
auction_bids.fillna(0,inplace = True)
auction_bids['percentage_of_auctions_above_threshold'] = auction_bids['no_of_auction_exceeds_threshold']/auction_bids['total_no_of_participated_auctions']
auction_bids

,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold
bidder_id,,,
001068c415025a009fee375a12cff4fcnht8y,1,0.0,0.000000
002d229ffb247009810828f648afc2ef593rb,1,0.0,0.000000
0030a2dd87ad2733e0873062e4f83954mkj86,1,0.0,0.000000
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3,0.0,0.000000
00486a11dff552c4bd7696265724ff81yeo9v,13,0.0,0.000000
...,...,...,...
ffbc0fdfbf19a8a9116b68714138f2902cc13,637,254.0,0.398744
ffc4e2dd2cc08249f299cab46ecbfacfobmr3,15,0.0,0.000000
ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1,0.0,0.000000


In [19]:
num_bidders_per_merchandise = bids_df.groupby("merchandise").nunique()
avg_num_bids_per_merchandise = num_bidders_per_merchandise['bid_id']/num_bidders_per_merchandise['bidder_id']
avg_num_bids_per_merchandise = avg_num_bids_per_merchandise.to_frame()
avg_num_bids_per_merchandise_df = avg_num_bids_per_merchandise.reset_index()
avg_num_bids_per_merchandise_df = avg_num_bids_per_merchandise_df.rename(columns={"merchandise": "merchandise", 0: "merchandise threshold"})
merchandise_df = bids_df.groupby(['bidder_id', 'merchandise']).size()
merchandise_df = pd.DataFrame(merchandise_df, columns = ['merchandise_bids_count'])
merchandise_df_reset = merchandise_df.reset_index()
merchandise_df_final = merchandise_df_reset.merge(avg_num_bids_per_merchandise_df, on='merchandise')

merchandise_df_final['exceeds_merchandise_bids_count_threshold'] = 1 * (merchandise_df_final['merchandise_bids_count'] > merchandise_df_final['merchandise threshold'])
total_no_of_bidded_merchandise_df = merchandise_df_final.groupby('bidder_id').nunique()
total_no_of_bidded_merchandise_df = total_no_of_bidded_merchandise_df[['merchandise']]
total_no_of_bidded_merchandise_df = total_no_of_bidded_merchandise_df.rename(columns={"merchandise":"total_no_of_bidded_category"})

bidders_above_merchandise_threshold = merchandise_df_final[merchandise_df_final['exceeds_merchandise_bids_count_threshold'] == 1]
bidders_above_merchandise_threshold_df = bidders_above_merchandise_threshold.groupby('bidder_id').nunique()
bidders_above_merchandise_threshold_df = bidders_above_merchandise_threshold_df[['merchandise']]
bidders_above_merchandise_threshold_df = bidders_above_merchandise_threshold_df.rename(columns={"merchandise": "no_of_merchandise_exceeds_threshold"})
merchandise_bids = total_no_of_bidded_merchandise_df.merge(bidders_above_merchandise_threshold_df, how= 'left', on = 'bidder_id')
merchandise_bids.fillna(0,inplace = True)
merchandise_bids['percentage_of_merchandise_above_threshold'] = merchandise_bids['no_of_merchandise_exceeds_threshold']/merchandise_bids['total_no_of_bidded_category']
merchandise_bids

,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold
bidder_id,,,
001068c415025a009fee375a12cff4fcnht8y,1,0.0,0.0
002d229ffb247009810828f648afc2ef593rb,1,0.0,0.0
0030a2dd87ad2733e0873062e4f83954mkj86,1,0.0,0.0
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1,0.0,0.0
00486a11dff552c4bd7696265724ff81yeo9v,1,0.0,0.0
...,...,...,...
ffbc0fdfbf19a8a9116b68714138f2902cc13,1,1.0,1.0
ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1,0.0,0.0
ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1,0.0,0.0


In [20]:
bot_or_human2 = train_df
url_used_by_bot = pd.merge(bids_df, bot_or_human2[['bidder_id', 'outcome']], on='bidder_id', how='left')

# #make outcome num a float num
url_used_by_bot['outcome'] = 1.0 * (url_used_by_bot['outcome'] == 1)

#get unique url by each bidder 
unique_url_per_bidder = url_used_by_bot.groupby(['url', 'bidder_id']).outcome.mean().reset_index() 

#identify url that are a bot
bot_url = unique_url_per_bidder.groupby('url').outcome.sum().reset_index() 
bot_url['num_bots_on_url'] = bot_url.outcome
bids2 = pd.merge(url_used_by_bot, bot_url[['url', 'num_bots_on_url']], on='url', how='left')

#find out bidders that are using url used by bots
bids2['has_url_used_by_a_bot'] = 1 * ((bids2['num_bots_on_url'] - bids2['outcome']) >= 1) 

#drop bidders who are already identified as bots in train dataset
bids2 = bids2.drop('outcome', 1)

#group by unique url and bidder
unique_url_per_bidder = bids2.groupby(['bidder_id', 'url']).has_url_used_by_a_bot.mean().reset_index() 

#get bidder that has url used by a bot
bot_url = unique_url_per_bidder.groupby('bidder_id').has_url_used_by_a_bot.mean().reset_index()

#final dataframe
bot_or_human2 = pd.merge(bot_or_human2, bot_url[['bidder_id', 'has_url_used_by_a_bot']], on='bidder_id', how='left')
bot_or_human2['on_url_that_has_a_bot_mean'] = bot_or_human2['has_url_used_by_a_bot'].fillna(0)
bot_or_human2['has_url_used_by_a_bot'] = 1*(bot_or_human2['has_url_used_by_a_bot'].fillna(0) > 0)
bot_or_human2 = bot_or_human2[['bidder_id', 'on_url_that_has_a_bot_mean']]
bot_or_human2

<ipython-input-20-8ce5221bb2bb>:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bids2 = bids2.drop('outcome', 1)


,bidder_id,on_url_that_has_a_bot_mean
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,1.000000
1,624f258b49e77713fc34034560f93fb3hu3jo,0.500000
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,0.500000
3,4bee9aba2abda51bf43d639013d6efe12iycd,1.000000
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.010989
...,...,...
2008,369515b3af4f8ca582f90271d30b14b6r52aw,0.500000
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,0.000000
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,0.000000
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,0.000000


In [21]:
# stdev of time per bid (skipz first)

In [22]:
# mean response time
def find_time_to_bid(auction_id, time):
    starttime = timing_dict[auction_id]['starttime']
    return int(time) - int(starttime)

end_auction_time= bids_df.groupby('auction').time.max().reset_index()
end_auction_time= end_auction_time.rename(columns= {'time': 'endtime'})

start_auction_time= bids_df.groupby('auction').time.min().reset_index()
start_auction_time= start_auction_time.rename(columns= {'time': 'starttime'})

start_end_times= pd.merge(start_auction_time, end_auction_time, on= 'auction', how= 'left')
start_end_times = start_end_times.set_index('auction')

timing_dict = start_end_times.to_dict('index')

start_auction_time= bids_df.groupby('auction').time.min().reset_index()
start_auction_time= start_auction_time.rename(columns= {'time': 'starttime'})
timing_dict = start_end_times.to_dict('index')

bids_df['time_to_bid'] = bids_df.apply(lambda row: find_time_to_bid(row['auction'], row['time']), axis=1)

bids_df_time = bids_df
bids_df_time_mean = bids_df_time.groupby('bidder_id')['time_to_bid'].mean().reset_index()
bids_df_time_mean

,bidder_id,time_to_bid
0,001068c415025a009fee375a12cff4fcnht8y,7.439058e+13
1,002d229ffb247009810828f648afc2ef593rb,1.455263e+11
2,0030a2dd87ad2733e0873062e4f83954mkj86,7.263689e+13
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,6.854544e+12
4,00486a11dff552c4bd7696265724ff81yeo9v,1.904773e+13
...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,5.719356e+12
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,2.954407e+13
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.044305e+13
6612,ffd62646d600b759a985d45918bd6f0431vmz,5.369726e+12


In [23]:
# number of simul bids per bidder
results = bids_df.groupby(['bidder_id','time']).count()[bids_df.groupby(['bidder_id','time']).count()['bid_id'] > 1 ]
inst_resp = results.groupby("bidder_id").sum()
needed_df = bids_df.groupby(['bidder_id','time']).count()[bids_df.groupby(['bidder_id','time']).count()['bid_id'] > 0 ]
num_resp = needed_df.groupby("bidder_id").sum()
num_resp = num_resp.reset_index()
inst_resp = inst_resp.reset_index()
num_resp = num_resp[['bidder_id','bid_id']]
inst_resp = inst_resp[['bidder_id','bid_id']]
num_resp.rename(columns={"bid_id": "num_bids"}, inplace= True)
inst_resp.rename(columns={"bid_id": "inst_resp"}, inplace= True)
total_inst_resp = num_resp.merge(inst_resp, how= 'left',on='bidder_id')
total_inst_resp.fillna(0, inplace = True)
total_inst_resp['perc_inst_resp'] = total_inst_resp['inst_resp']/total_inst_resp['num_bids']
total_inst_resp.drop(axis = 1 , columns = 'num_bids', inplace=True)
total_inst_resp

,bidder_id,inst_resp,perc_inst_resp
0,001068c415025a009fee375a12cff4fcnht8y,0.0,0.000000
1,002d229ffb247009810828f648afc2ef593rb,0.0,0.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,0.0,0.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,0.0,0.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,0.0,0.000000
...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,3068.0,0.122353
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,0.0,0.000000
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,0.0,0.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,0.0,0.000000


In [24]:
# one-hot encoding merchandise - working
one_hot = pd.get_dummies(bids_df['merchandise'])
one_hot_df = bids_df.join(one_hot)
one_hot_df = one_hot_df.drop('merchandise', axis=1)
# one_hot_df = one_hot_df.join(one_hot)
# one_hot_df = one_hot_df.rename(columns={1.0:'merchandise=1.0',2.0:'merchandise=2.0'})
one_hot_df.set_index("bidder_id", inplace=True)
one_hot_df = one_hot_df[['auto parts', 'books and music', 'clothing', 'computers',
       'furniture', 'home goods', 'jewelry', 'mobile', 'office equipment',
       'sporting goods']]
one_hot_df = one_hot_df.groupby("bidder_id").sum()
one_hot_df

,auto parts,books and music,clothing,computers,furniture,home goods,jewelry,mobile,office equipment,sporting goods
bidder_id,,,,,,,,,,
001068c415025a009fee375a12cff4fcnht8y,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
002d229ffb247009810828f648afc2ef593rb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
0030a2dd87ad2733e0873062e4f83954mkj86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
00486a11dff552c4bd7696265724ff81yeo9v,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
ffbc0fdfbf19a8a9116b68714138f2902cc13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25075.0,0.0,0.0
ffc4e2dd2cc08249f299cab46ecbfacfobmr3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0
ffd29eb307a4c54610dd2d3d212bf3bagmmpl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
# percent instant resp (skipz first)

In [26]:
# bids per auction
bids_auction_count= bids_df.groupby(["bidder_id", "auction"])
bids_auction_count_df = bids_auction_count.nunique().groupby("bidder_id").mean().reset_index()
bids_auction_count_df = pd.DataFrame(bids_auction_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                       'time', 'country', 'ip', 'url'])
bids_auction_count_df = bids_auction_count_df[['bidder_id','bid_id']]
bids_auction_count_df.rename(columns={"bid_id": "num_bids_per_auction"}, inplace= True)

In [27]:
# bids per device
bids_device_count= bids_df.groupby(["bidder_id", "device"])
bids_device_count_df = bids_device_count.nunique().groupby("bidder_id").mean().reset_index()
bids_device_count_df = pd.DataFrame(bids_device_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                     'time', 'country', 'ip', 'url'])
bids_device_count_df = bids_device_count_df[['bidder_id','bid_id']]
bids_device_count_df.rename(columns={"bid_id": "num_bids_per_device"}, inplace= True)

In [28]:
# bids per country
bids_country_count= bids_df.groupby(["bidder_id", "country"])
bids_country_count_df = bids_country_count.nunique().groupby("bidder_id").mean().reset_index()
bids_country_count_df = pd.DataFrame(bids_country_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                       'time', 'country', 'ip', 'url'])
bids_country_count_df = bids_country_count_df[['bidder_id','bid_id']]
bids_country_count_df.rename(columns={"bid_id": "num_bids_per_country"}, inplace= True)

In [29]:
# bids per ip
bids_ip_count= bids_df.groupby(["bidder_id", "ip"])
bids_ip_count_df = bids_ip_count.nunique().groupby("bidder_id").mean().reset_index()
bids_ip_count_df = pd.DataFrame(bids_ip_count_df, columns = ['bidder_id', 'bid_id', 'auction', 
                                                             'merchandise', 'time', 'country', 'ip', 'url'])
bids_ip_count_df = bids_ip_count_df[['bidder_id','bid_id']]
bids_ip_count_df.rename(columns={"bid_id": "num_bids_per_ip"}, inplace= True)

In [30]:
# bids per url
bids_url_count= bids_df.groupby(["bidder_id", "url"])
bids_url_count_df = bids_url_count.nunique().groupby("bidder_id").mean().reset_index()
bids_url_count_df = pd.DataFrame(bids_url_count_df, columns = ['bidder_id', 'bid_id', 'auction', 'merchandise', 
                                                                     'time', 'country', 'ip', 'url'])
bids_url_count_df = bids_url_count_df[['bidder_id','bid_id']]
bids_url_count_df.rename(columns={"bid_id": "num_bids_per_url"}, inplace= True)

In [31]:
merged_df = bids_auction_count_df.merge(bids_device_count_df, how= 'left', on = 'bidder_id')
merged_df = merged_df.merge(bids_country_count_df, how= 'left', on = 'bidder_id')
merged_df = merged_df.merge(bids_ip_count_df, how= 'left', on = 'bidder_id')
merged_df = pd.DataFrame(merged_df)
merged_df.fillna(0,inplace = True)
merged_df 

,bidder_id,num_bids_per_auction,num_bids_per_device,num_bids_per_country,num_bids_per_ip
0,001068c415025a009fee375a12cff4fcnht8y,1.000000,1.000000,1.000000,1.000000
1,002d229ffb247009810828f648afc2ef593rb,2.000000,1.000000,2.000000,2.000000
2,0030a2dd87ad2733e0873062e4f83954mkj86,1.000000,1.000000,1.000000,1.000000
3,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,1.000000,1.000000,3.000000,1.000000
4,00486a11dff552c4bd7696265724ff81yeo9v,1.538462,2.500000,20.000000,2.000000
...,...,...,...,...,...
6609,ffbc0fdfbf19a8a9116b68714138f2902cc13,39.364207,31.660354,245.833333,1.339047
6610,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,1.466667,1.692308,3.666667,1.222222
6611,ffd29eb307a4c54610dd2d3d212bf3bagmmpl,1.000000,1.000000,1.000000,1.000000
6612,ffd62646d600b759a985d45918bd6f0431vmz,12.072727,6.916667,664.000000,17.945946


#### Adding Statistical Features

In [32]:
# country per auction statistics
diversity_bids = bids_df.sort_values(['bidder_id','country'])
num_countries_per_auction = diversity_bids.groupby(['bidder_id','auction'])['country'].nunique()
num_countries_per_auction = pd.DataFrame(num_countries_per_auction)
mean_country_per_auction = num_countries_per_auction.groupby("bidder_id").mean()
mean_country_per_auction = mean_country_per_auction.reset_index().rename(columns = {'country':"mean_country_per_auction"})

max_country_per_auction = num_countries_per_auction.groupby("bidder_id").max()
max_country_per_auction = max_country_per_auction.reset_index().rename(columns = {'country':"max_country_per_auction"})

min_country_per_auction = num_countries_per_auction.groupby("bidder_id").min()
min_country_per_auction = min_country_per_auction.reset_index().rename(columns = {'country':"min_country_per_auction"})

std_country_per_auction = num_countries_per_auction.groupby("bidder_id").std()
std_country_per_auction = std_country_per_auction.reset_index().rename(columns = {'country':"std_country_per_auction"})

country_bids = pd.DataFrame(data = bids_df['bidder_id'].unique(), columns = ['bidder_id'],index = bids_df['bidder_id'].unique())

country_bids = country_bids.merge(mean_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(max_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(min_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(std_country_per_auction,on='bidder_id',how='left')

country_bids.fillna(0,inplace=True)
country_bids

,bidder_id,mean_country_per_auction,max_country_per_auction,min_country_per_auction,std_country_per_auction
0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,1.406326,69,1,3.582725
1,668d393e858e8126275433046bbd35c6tywop,2.318182,87,1,9.353004
2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,1.330037,69,1,3.278885
3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,1.335106,30,1,2.279751
4,8393c48eaf4b8fa96886edc7cf27b372dsibi,1.619318,37,1,3.084981
...,...,...,...,...,...
6609,57baadea0359cccd55d6e3b530b021f13umk8,1.000000,1,1,0.000000
6610,91c749114e26abdb9a4536169f9b4580huern,1.000000,1,1,0.000000
6611,a06311ea45de34bf667b79a92d13c764i6aiy,1.000000,1,1,0.000000
6612,dfb23124fba16b7778f65c1c8d091f949jz0k,1.000000,1,1,0.000000


In [33]:
# device per auction statistics
device_bids = bids_df.sort_values(['bidder_id','device'])
num_devices_per_auction = device_bids.groupby(['bidder_id','auction'])['device'].nunique()
num_devices_per_auction = pd.DataFrame(num_devices_per_auction)

mean_devices_per_auction = num_devices_per_auction.groupby("bidder_id").mean()
mean_devices_per_auction = mean_devices_per_auction.reset_index().rename(columns = {'device':"mean_devices_per_auction"})

max_devices_per_auction = num_devices_per_auction.groupby("bidder_id").max()
max_devices_per_auction = max_devices_per_auction.reset_index().rename(columns = {'device':"max_devices_per_auction"})

min_devices_per_auction = num_devices_per_auction.groupby("bidder_id").min()
min_devices_per_auction = min_devices_per_auction.reset_index().rename(columns = {'device':"min_devices_per_auction"})

std_devices_per_auction = num_devices_per_auction.groupby("bidder_id").std()
std_devices_per_auction = std_devices_per_auction.reset_index().rename(columns = {'device':"std_devices_per_auction"})

device_bids = pd.DataFrame(data = bids_df['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df['bidder_id'].unique())

device_bids = device_bids.merge(mean_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(max_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(min_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(std_devices_per_auction,on='bidder_id',how='left')
device_bids.fillna(0,inplace=True)
device_bids

,bidder_id,mean_devices_per_auction,max_devices_per_auction,min_devices_per_auction,std_devices_per_auction
0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,17.563260,513,1,44.188228
1,668d393e858e8126275433046bbd35c6tywop,11.094276,314,1,37.067386
2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,18.342398,325,1,31.516343
3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,8.148936,385,1,29.909295
4,8393c48eaf4b8fa96886edc7cf27b372dsibi,12.227273,486,1,38.901995
...,...,...,...,...,...
6609,57baadea0359cccd55d6e3b530b021f13umk8,1.000000,1,1,0.000000
6610,91c749114e26abdb9a4536169f9b4580huern,1.000000,1,1,0.000000
6611,a06311ea45de34bf667b79a92d13c764i6aiy,1.000000,1,1,0.000000
6612,dfb23124fba16b7778f65c1c8d091f949jz0k,1.000000,1,1,0.000000


In [34]:
# ip per auction statistics
ip_bids = bids_df.sort_values(['bidder_id','ip'])
num_ip_per_auction = ip_bids.groupby(['bidder_id','auction'])['ip'].nunique()
num_ip_per_auction = pd.DataFrame(num_ip_per_auction)

mean_ip_per_auction = num_ip_per_auction.groupby("bidder_id").mean()
mean_ip_per_auction = mean_ip_per_auction.reset_index().rename(columns = {'ip':"mean_ip_per_auction"})

max_ip_per_auction = num_ip_per_auction.groupby("bidder_id").max()
max_ip_per_auction = max_ip_per_auction.reset_index().rename(columns = {'ip':"max_ip_per_auction"})

min_ip_per_auction = num_ip_per_auction.groupby("bidder_id").min()
min_ip_per_auction = min_ip_per_auction.reset_index().rename(columns = {'ip':"min_ip_per_auction"})

std_ip_per_auction = num_ip_per_auction.groupby("bidder_id").std()
std_ip_per_auction = std_ip_per_auction.reset_index().rename(columns = {'ip':"std_ip_per_auction"})

ip_bids = pd.DataFrame(data = bids_df['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df['bidder_id'].unique())

ip_bids = ip_bids.merge(mean_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(max_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(min_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(std_ip_per_auction,on='bidder_id',how='left')

ip_bids.fillna(0,inplace=True)
ip_bids

,bidder_id,mean_ip_per_auction,max_ip_per_auction,min_ip_per_auction,std_ip_per_auction
0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,37.017032,4748,1,205.728952
1,668d393e858e8126275433046bbd35c6tywop,24.377104,1134,1,120.223888
2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,78.381953,9218,1,426.443569
3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,37.558511,4320,1,317.646731
4,8393c48eaf4b8fa96886edc7cf27b372dsibi,56.403409,6994,1,527.253240
...,...,...,...,...,...
6609,57baadea0359cccd55d6e3b530b021f13umk8,1.000000,1,1,0.000000
6610,91c749114e26abdb9a4536169f9b4580huern,1.000000,1,1,0.000000
6611,a06311ea45de34bf667b79a92d13c764i6aiy,1.000000,1,1,0.000000
6612,dfb23124fba16b7778f65c1c8d091f949jz0k,1.000000,1,1,0.000000


In [35]:
# url per auction statistics
url_bids= bids_df.sort_values(['bidder_id','url'])
num_url_per_auction = url_bids.groupby(['bidder_id','auction'])['url'].nunique()
num_url_per_auction = pd.DataFrame(num_url_per_auction)

mean_url_per_auction = num_url_per_auction.groupby("bidder_id").mean()
mean_url_per_auction = mean_url_per_auction.reset_index().rename(columns = {'url':"mean_url_per_auction"})

max_url_per_auction = num_url_per_auction.groupby("bidder_id").max()
max_url_per_auction = max_url_per_auction.reset_index().rename(columns = {'url':"max_url_per_auction"})

min_url_per_auction = num_url_per_auction.groupby("bidder_id").min()
min_url_per_auction = min_url_per_auction.reset_index().rename(columns = {'url':"min_url_per_auction"})

std_url_per_auction = num_url_per_auction.groupby("bidder_id").std()
std_url_per_auction = std_url_per_auction.reset_index().rename(columns = {'url':"std_url_per_auction"})

url_bids = pd.DataFrame(data = bids_df['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df['bidder_id'].unique())

url_bids = url_bids.merge(mean_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(max_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(min_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(std_url_per_auction,on='bidder_id',how='left')

url_bids.fillna(0,inplace=True)
url_bids

,bidder_id,mean_url_per_auction,max_url_per_auction,min_url_per_auction,std_url_per_auction
0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,31.708029,1739,1,124.022963
1,668d393e858e8126275433046bbd35c6tywop,11.947811,555,1,57.870813
2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,17.912237,2665,1,112.575095
3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,23.489362,1987,1,149.267713
4,8393c48eaf4b8fa96886edc7cf27b372dsibi,16.153409,1357,1,103.183633
...,...,...,...,...,...
6609,57baadea0359cccd55d6e3b530b021f13umk8,1.000000,1,1,0.000000
6610,91c749114e26abdb9a4536169f9b4580huern,1.000000,1,1,0.000000
6611,a06311ea45de34bf667b79a92d13c764i6aiy,1.000000,1,1,0.000000
6612,dfb23124fba16b7778f65c1c8d091f949jz0k,1.000000,1,1,0.000000


#### Merging all features together again

In [36]:
merged_df_2 = new_train.merge(bids_df_time_mean, how= 'left',on='bidder_id') # mean response time
merged_df_2 = merged_df_2.merge(total_inst_resp, how= 'left',on='bidder_id') # number of simul bids per bidder
merged_df_2 = merged_df_2.merge(one_hot_df, how= 'left',on='bidder_id') # one-hot encoding merchandise
merged_df_2 = merged_df_2.merge(merged_df, how= 'left',on='bidder_id') # num bids per xxx per bidder id
merged_df_2 = merged_df_2.merge(bot_or_human, how= 'left',on='bidder_id') # fraction of IPs used by a bidder which were also used by another user which was a bot
merged_df_2 = merged_df_2.merge(entropy_df, how= 'left',on='bidder_id') # entropy
merged_df_2 = merged_df_2.merge(country_bids, how= 'left',on='bidder_id') # country per auction statistics
merged_df_2 = merged_df_2.merge(device_bids, how= 'left',on='bidder_id') # device per auction statistics
merged_df_2 = merged_df_2.merge(ip_bids, how= 'left',on='bidder_id') # ip per auction statistics
merged_df_2 = merged_df_2.merge(url_bids, how= 'left',on='bidder_id') # url per auction statistics
merged_df_2 = merged_df_2.merge(auction_bids, how= 'left',on='bidder_id') #bidders that exceeded avg bids per auction
merged_df_2 = merged_df_2.merge(merchandise_bids, how= 'left',on='bidder_id') #bidders that exceeded avg bids per merchandise
merged_df_2 = merged_df_2.merge(bot_or_human2, how= 'left',on='bidder_id') # fraction of urls used by a bidder which were also used by another user which was a bot

new_train = merged_df_2
new_train.fillna(0,inplace = True)

new_train

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,20.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,3.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,4.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,123.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,5.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,2.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [37]:
merged_df_3 = new_test.merge(bids_df_time_mean, how= 'left',on='bidder_id') # mean response time
merged_df_3 = merged_df_3.merge(total_inst_resp, how= 'left',on='bidder_id') # number of simul bids per bidder
merged_df_3 = merged_df_3.merge(one_hot_df, how= 'left',on='bidder_id') # one-hot encoding merchandise
merged_df_3 = merged_df_3.merge(merged_df, how= 'left',on='bidder_id') # num bids per xxx per bidder id
merged_df_3 = merged_df_3.merge(bot_or_human, how= 'left',on='bidder_id') # fraction of IPs used by a bidder which were also used by another user which was a bot
merged_df_3 = merged_df_3.merge(entropy_df, how= 'left',on='bidder_id') # entropy
merged_df_3 = merged_df_3.merge(country_bids, how= 'left',on='bidder_id') # country per auction statistics
merged_df_3 = merged_df_3.merge(device_bids, how= 'left',on='bidder_id') # device per auction statistics
merged_df_3 = merged_df_3.merge(ip_bids, how= 'left',on='bidder_id') # ip per auction statistics
merged_df_3 = merged_df_3.merge(url_bids, how= 'left',on='bidder_id') # url per auction statistics
merged_df_3 = merged_df_3.merge(auction_bids, how= 'left',on='bidder_id') #bidders that exceeded avg bids per auction
merged_df_3 = merged_df_3.merge(merchandise_bids, how= 'left',on='bidder_id') #bidders that exceeded avg bids per merchandise
merged_df_3 = merged_df_3.merge(bot_or_human2, how= 'left',on='bidder_id') # fraction of urls used by a bidder which were also used by another user which was a bot

new_test = merged_df_3
new_test.fillna(0,inplace = True)

new_test

,bidder_id,payment_account,address,auction,merchandise,device,time,country,ip,url,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,49bb5a3c944b8fc337981cc7a9ccae41u31d7,a3d2de7675556553a5f08e4c88d2c228htx90,5d9fa1b71f992e7c7a106ce4b07a0a754le7c,3.0,1.0,2.0,4.0,3.0,4.0,3.0,...,2.0,1.0,0.577350,3.0,0.0,0.000000,1.0,0.0,0.0,0.0
1,a921612b85a1494456e74c09393ccb65ylp4y,a3d2de7675556553a5f08e4c88d2c228rs17i,a3d2de7675556553a5f08e4c88d2c228klidn,2.0,1.0,3.0,3.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,2.0,0.0,0.000000,1.0,0.0,0.0,0.0
2,6b601e72a4d264dab9ace9d7b229b47479v6i,925381cce086b8cc9594eee1c77edf665zjpl,a3d2de7675556553a5f08e4c88d2c228aght0,14.0,1.0,4.0,17.0,3.0,4.0,2.0,...,2.0,1.0,0.267261,14.0,0.0,0.000000,1.0,0.0,0.0,0.0
3,eaf0ed0afc9689779417274b4791726cn5udi,a3d2de7675556553a5f08e4c88d2c228nclv5,b5714de1fd69d4a0d2e39d59e53fe9e15vwat,90.0,1.0,81.0,148.0,14.0,129.0,80.0,...,8.0,1.0,0.988132,90.0,0.0,0.000000,1.0,0.0,0.0,0.0
4,cdecd8d02ed8c6037e38042c7745f688mx5sf,a3d2de7675556553a5f08e4c88d2c228dtdkd,c3b363a3c3b838d58c85acf0fc9964cb4pnfa,20.0,1.0,17.0,23.0,2.0,17.0,1.0,...,1.0,1.0,0.000000,20.0,0.0,0.000000,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4695,bef56983ba78b2ee064443ae95972877jfkyd,0f235a6dfea5a5885d63968826b748b4q4dra,a98a4841db165de919d29cb49d0bc306cq21h,41.0,1.0,9.0,466.0,5.0,22.0,4.0,...,3.0,1.0,0.512050,41.0,13.0,0.317073,1.0,1.0,1.0,0.0
4696,4da45cc915c32d4368ac7e773d92d4affwqrr,9e0adf7481c422654d4d0a849e0e50abiumen,e23d9777cddc347de82d839b2e54b22ecopkp,32.0,1.0,29.0,66.0,10.0,49.0,18.0,...,8.0,1.0,1.364513,32.0,1.0,0.031250,1.0,0.0,0.0,0.0
4697,0d0e6220bf59ab9a0c5b5987fb2c34a9p33f9,7df4ebd184668b4257f740b11d4519afq7kr1,b650404e1ab5d177020221277c3e9306qegyl,419.0,1.0,376.0,2156.0,86.0,1460.0,1049.0,...,103.0,1.0,7.248891,419.0,24.0,0.057279,1.0,1.0,1.0,0.0
4698,4981c32c54dde65b79dbc48fd9ab6457caqze,a3d2de7675556553a5f08e4c88d2c2284qlm0,9c35320088eaf32046a51a96ebb2e658i479u,5.0,1.0,4.0,5.0,1.0,5.0,2.0,...,1.0,1.0,0.000000,5.0,0.0,0.000000,1.0,0.0,0.0,0.0


#### Scaling the values

In [38]:
new_train.to_csv("new_train.csv")
new_test.to_csv("new_test.csv")